# Курсовая работа
 Выполнила:
 студента группы БФИ2202
 Красько А.Л.

 Красько А.Л.

In [231]:
import csv

CAPACITY = 10
PATH = 'sales_data.csv'


Сохраняем данные в хэш-таблицу:

In [232]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None


class HashTable: 
    def __init__(self, capacity):
        self.capacity = capacity
        self.size = 0
        self.table = [None] * capacity

    def _hash(self, key):
        return hash(key) % self.capacity
    
    def insert(self, key, value):
        index = self._hash(key)
        if self.table[index] is None:
            self.table[index] = Node(key, value)
            self.size += 1
        else:
            current = self.table[index]
            while current: 
                if current.key == key:
                    current.value = value
                    return
                current = current.next
            new_node = Node(key, value)
            new_node.next = self.table[index]
            self.table[index] = new_node
            self.size += 1
            if self.size > 0.8 * self.capacity:
                self.resize()
        
    def search(self, key):
        index = self._hash(key)
        current = self.table[index]
        while current:
            if current.key == key:
                return current.value
            current = current.next
        raise KeyError(key)

    def resize(self):
        new_capacity = self.capacity * 2
        new_table = [None] * new_capacity
        for i in range(self.capacity):
            node = self.table[i]
            while node:
                index = self._hash(node.key)
                if new_table[index]:
                    new_node = new_table[index]
                    while new_node.next:
                        new_node = new_node.next
                    new_node.next = Node(node.key, node.value)
                else:
                    new_table[index] = Node(node.key, node.value)
                node = node.next
        self.capacity = new_capacity
        self.table = new_table
    
    def get_table(self): 
        elements = [] 
        for i in range(self.capacity): 
            current = self.table[i] 
            while current: 
                elements.append((current.key, current.value)) 
                current = current.next
        return elements

In [233]:
def import_data(file_path: str, data: HashTable):
    """Импорт данных из cvs файла."""
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.insert(row['Название товара'], {'quantity': float(row['Количество продаж']), 'price': float(row['Цена за единицу'])})
    return data


In [234]:
def quick_sort(array: list, sort_by):
    """Быстрая сортировка."""
    if len(array) <= 1:
        return array
    middle_element_index = len(array) // 2
    if sort_by == 'revenue':
        pivot = array[middle_element_index][1]['quantity'] * array[middle_element_index][1]['price']
    else:
        pivot = array[middle_element_index][0]
    left, center, right = partition(array, pivot, sort_by)
    return quick_sort(left, sort_by=sort_by) + center + quick_sort(right, sort_by=sort_by)

def partition(array, pivot, sort_by):
    left, center, right = [], [], []
    for item in array:
        if sort_by == 'revenue':
            parametr = item[1]['quantity']*item[1]['price']
        else:
            parametr = item[0]
        if parametr > pivot:
            left.append(item)
        elif parametr < pivot:
            right.append(item)
        elif parametr == pivot:
            center.append(item)
    return left, center, right


In [245]:
def bin_search(arr, product_name):
    length = len(arr)
    data = quick_sort(arr, sort_by='name')
    print(*data, sep='\n')
    left_point = 0
    right_point = length - 1
    while left_point < right_point:
        mid = (left_point + right_point) // 2
        print(data[mid][0])
        if data[mid][0] > product_name:
            left_point = mid + 1
        elif data[mid][0] == product_name:
            print(f'\nТовар {product_name} найден! {data[mid]}')
            return data[mid]
        else:
            right_point = mid - 1
    return 'Товар не найден'

In [236]:
def calculate_revenue(data: list):
    sorted_data = quick_sort(data, sort_by='revenue')
    total_revenue = 0
    max_quantity = 0
    print('Товар, отсортированный по выруxrt')
    for item in sorted_data:
        revenue = item[1]['quantity']*item[1]['price']
        total_revenue += revenue
        if item[1]['quantity'] > max_quantity:
            max_quantity = item[1]['quantity']
        print(item, f' revenue: {revenue:.2f}')
    max_revenue = sorted_data[0][1]['quantity']*sorted_data[0][1]['price']


In [247]:
data = import_data(PATH, HashTable(CAPACITY)).get_table()
bin_search(data, 'WidgetC')


('WidgetC', {'quantity': 20.0, 'price': 22.99})
('WidgetA', {'quantity': 25.0, 'price': 19.99})
('ThingamajigD', {'quantity': 12.0, 'price': 27.99})
('ThingamajigB', {'quantity': 27.0, 'price': 24.99})
('TableB', {'quantity': 3.0, 'price': 149.99})
('SofaE', {'quantity': 2.0, 'price': 599.99})
('RugD', {'quantity': 7.0, 'price': 79.99})
('LampC', {'quantity': 12.0, 'price': 29.99})
('GizmoZ', {'quantity': 32.0, 'price': 14.99})
('GizmoA', {'quantity': 15.0, 'price': 12.99})
('GadgetY', {'quantity': 30.0, 'price': 19.99})
('GadgetX', {'quantity': 18.0, 'price': 29.99})
('DoohickeyZ', {'quantity': 22.0, 'price': 14.99})
('DoohickeyY', {'quantity': 41.0, 'price': 9.99})
('ChairA', {'quantity': 5.0, 'price': 89.99})
LampC
ThingamajigB
WidgetA


'Товар не найден'